In [1]:
import numpy as np
import pandas as pd
from portfolio_advisor import easy_add_assets

In [38]:
from load_data import Balance, Instruments, AdvisedPortfolios, PriceDB, Singleton
from utils import get_current_pos, get_advised_port, get_recommendation

In [39]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
userid = 'A01'

In [41]:
old_port = get_current_pos(userid=userid)
new_port = get_advised_port(risk_profile=2)

In [42]:
old_new = pd.merge(old_port.loc[:, ['exposure', 'itemcode', 'wt']], new_port.loc[:, ['exposure', 'itemcode', 'wt']],
                left_on=['exposure'], right_on=['exposure'], how='outer', suffixes=['_old', '_new'])
old_new = old_new.fillna(value={'wt_old':0, 'wt_new':0})
old_new['wt_diff'] = np.abs(old_new['wt_new'] - old_new['wt_old'])

In [43]:
old_new.loc[3, 'exposure'] = 'EQ_M_KR_LARGE'

In [44]:
exp_grp = old_new[old_new.exposure=='EQ_M_KR_LARGE']

In [79]:
exp_grp

,exposure,itemcode_old,wt_old,itemcode_new,wt_new,wt_diff
2,EQ_M_KR_LARGE,A069500,0.060862,A102110,0.03,0.030862
3,EQ_M_KR_LARGE,A360200,0.029569,NaN,0.00,0.029569


In [76]:
import math
import itertools

In [74]:
olds = exp_grp.loc[exp_grp.itemcode_old.isna()==False, 'itemcode_old'].tolist()
news = exp_grp.loc[exp_grp.itemcode_new.isna()==False, 'itemcode_new'].tolist()


In [78]:
list(itertools.product(olds, news))

[('A069500', 'A102110'), ('A360200', 'A102110')]

In [11]:
oldcode_idx = exp_grp.wt_diff.idxmin()

In [18]:
exp_grp.loc[oldcode_idx, 'itemcode_new'] = exp_grp.loc[oldcode_idx, 'itemcode_old']

C:\Users\Administrator\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [19]:
exp_grp.loc[oldcode_idx, 'itemcode_old']

'A360200'

In [22]:
 exp_grp.loc[oldcode_idx, 'itemcode_old']

'A360200'

In [15]:
exp_grp.itemcode_new

2    A102110
3        NaN
Name: itemcode_new, dtype: object

In [23]:
oldcode_idx

3

In [ ]:
def reduce_obvious_costs(old_port, new_port):
    r"""
    현재 포트폴리오 구성(old_port) 추천 포트폴리오 구성(new_port)를 입력받아
    불필요한 매매비용을 줄이도록 종목코드를 변경한다.
    
    예를 들어 KINDEX 200 10% -> KODEX 200 15%로 바꾸는 것은 매도 10%, 매수 15%를 해야 된다.
    이 둘은 사실상 같은 투자효과를 내므로 EM_M_KR_LARGE라는 exposure로 분류되어 있으므로,
    KINDEX 10% -> KINDEX 15%로 바꿔 생각하면 매수 5%만 해도 된다.
    따라서 20%에 대한 매매비용이 절약된다. 이는 최적화라기보다는 명백한 잘못을 하지 말아야 되는
    기본 기능으로 분류된다.
    
    Parameters
    ----------
    old_port : pd.DataFrame
        현재 포트폴리오. 컬럼으로 ['exposure', 'itemcode', 'wt']가 필요함.
    new_port : pd.DataFrame
        추천(미래) 포트폴리오. 컬럼으로 ['exposure', 'itemcode', 'wt']가 필요함.
        
    Return
    ------
    pd.DataFrame
    불필요한 매매비용이 줄도록 new_port의 itemcode를 적절히 변경하고,
    old_port와 new_port를 병합하여 1개의 DataFrame 인스턴스를 반환.
    """
    
    # 두 포트폴리오 병합하고 & nan 처리
    old_new = pd.merge(old_port.loc[:, ['exposure', 'itemcode', 'wt']], new_port.loc[:, ['exposure', 'itemcode', 'wt']],
                    left_on=['exposure'], right_on=['exposure'], how='outer', suffixes=['_old', '_new'])
    old_new = old_new.fillna(value={'wt_old':0, 'wt_new':0})
    
    
    old_new['wt_diff'] = np.abs(old_new['wt_new'] - old_new['wt_old'])
    
    for exp in set(old_new.exposure):
        exp_grp = old_new[old_new.exposure==exp]
        
        # 같은 exposure 그룹 내에서,
        # 비중차이(wt_diff)를 가장 작게 만드는 old_port 종목코드(itemcode) 인덱스를 선택한다
        oldcode_idx = exp_grp.wt_diff.idxmin()
        
        exp_grp[exp_grp.itemcode_new==exp_grp.loc[oldcode_idx, 'itemcode_old']]
        

In [127]:
exp_grp

,exposure,itemcode_old,wt_old,itemcode_new,wt_new
2,EQ_M_KR_LARGE,A069500,0.060862,A102110,0.03
3,EQ_M_KR_LARGE,A360200,0.029569,NaN,0.00


In [128]:
oldcode_idx = exp_grp.wt_diff.idxmin()

AttributeError: 'DataFrame' object has no attribute 'wt_diff'

In [65]:
for exp in set(old_new.exposure):
    exp_grp = old_new[old_new.exposure==exp]
    min_idx = exp_grp.wt_diff.idxmin()
    
    # 같은 exposure인 row만 추출해서(예: EQ_M_KR_LARGE. 한국시장대형주)
    # Case 별로 나누어 따져 본다.
    
    # Case 1: old에 1개 or 초과, new에 없음 -> pass.
    # Case 2: old에 없음, new에 1개 or 초과 -> pass.
    if exp_grp.itemcode_old.isnull().all() or exp_grp.itemcode_new.isnull().all():
        pass:

    
    elif:
    

    exposure itemcode_old    wt_old itemcode_new  wt_new   wt_diff
7  FI_D_LONG      A302190  0.073483          NaN     0.0  0.073483
    exposure itemcode_old    wt_old itemcode_new  wt_new   wt_diff
8  AL_FX_USD      A329750  0.109372          NaN     0.0  0.109372
  exposure itemcode_old    wt_old itemcode_new  wt_new   wt_diff
1  DEPOSIT      D000001  0.501074          NaN     0.0  0.501074
  exposure itemcode_old    wt_old itemcode_new  wt_new   wt_diff
0     CASH      C000001  0.027733          NaN     0.0  0.027733
      exposure itemcode_old  wt_old itemcode_new    wt_new   wt_diff
10  FI_D_SHORT          NaN     0.0      A122260  0.250000  0.250000
11  FI_D_SHORT          NaN     0.0      A114260  0.152149  0.152149
      exposure itemcode_old    wt_old itemcode_new  wt_new   wt_diff
9  EQ_S_NASDAQ      A367380  0.044114          NaN     0.0  0.044114
  exposure itemcode_old    wt_old itemcode_new    wt_new   wt_diff
4       MM      A196230  0.153792      A130730  0.250000  0.

In [90]:
exp_grp = old_new[old_new.exposure=='FI_D_LONG']

In [95]:
exp_grp.itemcode_old.isnull().all()

False

In [96]:
exp_grp.itemcode_new.isnull().all()

True

In [97]:
exp_grp

,exposure,itemcode_old,wt_old,itemcode_new,wt_new,wt_diff
7,FI_D_LONG,A302190,0.073483,NaN,0.0,0.073483


In [86]:
min_idx = exp_grp.wt_diff.idxmin()

In [87]:
exp_grp.loc[min_idx, 'itemcode_new'] = exp_grp.loc[min_idx, 'itemcode_old']

In [98]:
old_new

,exposure,itemcode_old,wt_old,itemcode_new,wt_new,wt_diff
0,CASH,C000001,0.027733,NaN,0.000000,0.027733
1,DEPOSIT,D000001,0.501074,NaN,0.000000,0.501074
2,EQ_M_KR_LARGE,A069500,0.060862,A102110,0.030000,0.030862
3,EQ_M_US_LARGE,A360200,0.029569,NaN,0.000000,0.029569
4,MM,A196230,0.153792,A130730,0.250000,0.096208
5,MM,A196230,0.153792,A153130,0.217851,0.064059
6,MM,A196230,0.153792,A214980,0.100000,0.053792
7,FI_D_LONG,A302190,0.073483,NaN,0.000000,0.073483
8,AL_FX_USD,A329750,0.109372,NaN,0.000000,0.109372
9,EQ_S_NASDAQ,A367380,0.044114,NaN,0.000000,0.044114


In [7]:
new_port

,date,itemcode,wt,tracking_code,itemname,exposure,price,volume,trading_amt_mln,itemtype
0,2021-02-26,A122260,0.250000,BOK_D1,KOSEF 통안채1년,FI_D_SHORT,101155.0,829489.0,61672.391133,ETF
1,2021-02-26,A130730,0.250000,MM,KOSEF 단기자금,MM,100960.0,965560.0,64369.329522,ETF
2,2021-02-26,A153130,0.217851,MM,KODEX 단기채권,MM,102725.0,39030.0,2884.683672,ETF
3,2021-02-26,A114260,0.152149,KTB_D3,KODEX 국고채3년,FI_D_SHORT,58090.0,5062.0,821.950469,ETF
4,2021-02-26,A214980,0.100000,MM_PLUS,KODEX 단기채권PLUS,MM,102995.0,9495.0,80590.448314,ETF
5,2021-02-26,A102110,0.030000,K200,TIGER 200,EQ_M_KR_LARGE,41540.0,1256046.0,77215.429155,ETF


In [ ]:
old_port.loc[:]

In [8]:
pd.merge(old_port, new_port, left_on=['exposure'], right_on=['exposure'], how='outer', suffixes=['_old', '_new'])

,date_old,itemcode_old,itemname_old,quantity,price_old,value,wt_old,tracking_code_old,exposure,date_new,itemcode_new,wt_new,tracking_code_new,itemname_new,price_new,volume,trading_amt_mln,itemtype
0,2/11/2020 9:00:00 AM,C000001,현금,19565.0,1.00,19565.00,0.027733,CASH,CASH,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2/11/2020 9:00:00 AM,D000001,예금,350000.0,1.01,353500.00,0.501074,DEPOSIT,DEPOSIT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2/11/2020 9:00:00 AM,A069500,KODEX 200,1.0,42936.90,42936.90,0.060862,K200,EQ_M_KR_LARGE,2021-02-26,A102110,0.030000,K200,TIGER 200,41540.0,1256046.0,77215.429155,ETF
3,2/11/2020 9:00:00 AM,A360200,KINDEX 미국S&P500,2.0,10430.40,20860.80,0.029569,SPX,EQ_M_US_LARGE,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2/11/2020 9:00:00 AM,A196230,KBSTAR 단기통안채,1.0,108498.00,108498.00,0.153792,BOK_D.4,MM,2021-02-26,A130730,0.250000,MM,KOSEF 단기자금,100960.0,965560.0,64369.329522,ETF
5,2/11/2020 9:00:00 AM,A196230,KBSTAR 단기통안채,1.0,108498.00,108498.00,0.153792,BOK_D.4,MM,2021-02-26,A153130,0.217851,MM,KODEX 단기채권,102725.0,39030.0,2884.683672,ETF
6,2/11/2020 9:00:00 AM,A196230,KBSTAR 단기통안채,1.0,108498.00,108498.00,0.153792,BOK_D.4,MM,2021-02-26,A214980,0.100000,MM_PLUS,KODEX 단기채권PLUS,102995.0,9495.0,80590.448314,ETF
7,2/11/2020 9:00:00 AM,A302190,TIGER 중장기국채,1.0,51841.35,51841.35,0.073483,KTB_D5,FI_D_LONG,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2/11/2020 9:00:00 AM,A329750,TIGER 미국달러단기채권액티브,8.0,9645.00,77160.00,0.109372,UST_D.5,AL_FX_USD,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2/11/2020 9:00:00 AM,A367380,KINDEX 미국나스닥100,3.0,10374.00,31122.00,0.044114,NQ100,EQ_S_NASDAQ,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
new_port

,date,itemcode,wt,tracking_code,itemname,exposure,price,volume,trading_amt_mln,itemtype
0,2021-02-26,A122260,0.250000,BOK_D1,KOSEF 통안채1년,FI_D_SHORT,101155.0,829489.0,61672.391133,ETF
1,2021-02-26,A130730,0.250000,MM,KOSEF 단기자금,MM,100960.0,965560.0,64369.329522,ETF
2,2021-02-26,A153130,0.217851,MM,KODEX 단기채권,MM,102725.0,39030.0,2884.683672,ETF
3,2021-02-26,A114260,0.152149,KTB_D3,KODEX 국고채3년,FI_D_SHORT,58090.0,5062.0,821.950469,ETF
4,2021-02-26,A214980,0.100000,MM_PLUS,KODEX 단기채권PLUS,MM,102995.0,9495.0,80590.448314,ETF
5,2021-02-26,A102110,0.030000,K200,TIGER 200,EQ_M_KR_LARGE,41540.0,1256046.0,77215.429155,ETF


In [39]:
from asset import Asset
from price import Price
from portfolio import Portfolio

In [11]:
# Assets in portfolio
# The price will be retrieved automatically
tickers = ["XBB.TO",   # iShares Core Canadian Universe Bond Index ETF
           "XIC.TO",   # iShares Core S&P/TSX Capped Composite Index ETF
           "ITOT",     # iShares Core S&P Total U.S. Stock Market ETF
           "IEFA",     # iShares Core MSCI EAFE ETF
           "IEMG"]     # iShares Core MSCI Emerging Markets ETF
quantities = [36, 64, 32, 8, 7]
prices = [100,120,110,95,10]

In [12]:
old_alloc = easy_add_assets(tickers=tickers, quantities=quantities, prices=prices)

In [13]:
old_alloc['IEFA'].quantity

8

In [17]:
old_port

,date,itemcode,itemname,quantity,price,value,wt,tracking_code,exposure
0,2/11/2020 9:00:00 AM,C000001,현금,19565,1.00,19565.00,0.027733,CASH,CASH
1,2/11/2020 9:00:00 AM,D000001,예금,350000,1.01,353500.00,0.501074,DEPOSIT,DEPOSIT
2,2/11/2020 9:00:00 AM,A069500,KODEX 200,1,42936.90,42936.90,0.060862,K200,EQ_M_KR_LARGE
3,2/11/2020 9:00:00 AM,A360200,KINDEX 미국S&P500,2,10430.40,20860.80,0.029569,SPX,EQ_M_US_LARGE
4,2/11/2020 9:00:00 AM,A196230,KBSTAR 단기통안채,1,108498.00,108498.00,0.153792,BOK_D.4,MM
5,2/11/2020 9:00:00 AM,A302190,TIGER 중장기국채,1,51841.35,51841.35,0.073483,KTB_D5,FI_D_LONG
6,2/11/2020 9:00:00 AM,A329750,TIGER 미국달러단기채권액티브,8,9645.00,77160.00,0.109372,UST_D.5,AL_FX_USD
7,2/11/2020 9:00:00 AM,A367380,KINDEX 미국나스닥100,3,10374.00,31122.00,0.044114,NQ100,EQ_S_NASDAQ


In [40]:
old_tickers = old_port.exposure.tolist()
old_quantities = old_port.quantity.tolist()
old_prices = old_port.price.tolist()



In [23]:
old_alloc = easy_add_assets(tickers=old_tickers, quantities=old_quantities, prices=old_prices)

In [24]:
old_alloc

{'CASH': <asset.Asset at 0x25516bb81c0>,
 'DEPOSIT': <asset.Asset at 0x25516bb8160>,
 'EQ_M_KR_LARGE': <asset.Asset at 0x25516e68430>,
 'EQ_M_US_LARGE': <asset.Asset at 0x25516e68130>,
 'MM': <asset.Asset at 0x25516e681c0>,
 'FI_D_LONG': <asset.Asset at 0x25516e68280>,
 'AL_FX_USD': <asset.Asset at 0x25516e68310>,
 'EQ_S_NASDAQ': <asset.Asset at 0x25516e683d0>}

In [32]:
p = Portfolio()

In [41]:
p.easy_add_assets(tickers=old_tickers, quantities=old_quantities, prices=old_prices)

{'CASH': <asset.Asset at 0x25516e68550>,
 'DEPOSIT': <asset.Asset at 0x25516e688e0>,
 'EQ_M_KR_LARGE': <asset.Asset at 0x25516e68c10>,
 'EQ_M_US_LARGE': <asset.Asset at 0x25516e689a0>,
 'MM': <asset.Asset at 0x25516e68a30>,
 'FI_D_LONG': <asset.Asset at 0x25516e68b20>,
 'AL_FX_USD': <asset.Asset at 0x25516f68430>,
 'EQ_S_NASDAQ': <asset.Asset at 0x25516f68100>}

In [42]:
p.selling_allowed = True

In [48]:
(new_port.wt*100).tolist()

[24.99999999983805,
 24.9999999957163,
 21.785129484296647,
 15.214870510294329,
 10.000000000386107,
 2.9999999987254866]

In [49]:
new_tickers = new_port.exposure.tolist()
new_quantities = (new_port.wt*100).tolist()     # 단위가 %이므로 100을 곱한다.

In [53]:
new_tickers

['FI_D_SHORT', 'MM', 'MM', 'FI_D_SHORT', 'MM', 'EQ_M_KR_LARGE']

In [51]:
dict(zip(new_tickers, new_quantities))    

{'FI_D_SHORT': 15.214870510294329,
 'MM': 10.000000000386107,
 'EQ_M_KR_LARGE': 2.9999999987254866}

In [40]:
new_port

,date,itemcode,wt,tracking_code,itemname,exposure
0,2021-02-26,A122260,0.250000,BOK_D1,KOSEF 통안채1년,FI_D_SHORT
1,2021-02-26,A130730,0.250000,MM,KOSEF 단기자금,MM
2,2021-02-26,A153130,0.217851,MM,KODEX 단기채권,MM
3,2021-02-26,A114260,0.152149,KTB_D3,KODEX 국고채3년,FI_D_SHORT
4,2021-02-26,A214980,0.100000,MM_PLUS,KODEX 단기채권PLUS,MM
5,2021-02-26,A102110,0.030000,K200,TIGER 200,EQ_M_KR_LARGE


In [28]:
price_db = PriceDB.instance().data

In [30]:
new_port.merge(price_db, left_on=['date', 'itemcode'], right_on=['date', 'itemcode'], how='left').drop(['ret'], axis=1)

,date,itemcode,wt,tracking_code,itemname,exposure,price,volume,trading_amt_mln,itemtype
0,2021-02-26,A122260,0.250000,BOK_D1,KOSEF 통안채1년,FI_D_SHORT,101155.0,829489.0,61672.391133,ETF
1,2021-02-26,A130730,0.250000,MM,KOSEF 단기자금,MM,100960.0,965560.0,64369.329522,ETF
2,2021-02-26,A153130,0.217851,MM,KODEX 단기채권,MM,102725.0,39030.0,2884.683672,ETF
3,2021-02-26,A114260,0.152149,KTB_D3,KODEX 국고채3년,FI_D_SHORT,58090.0,5062.0,821.950469,ETF
4,2021-02-26,A214980,0.100000,MM_PLUS,KODEX 단기채권PLUS,MM,102995.0,9495.0,80590.448314,ETF
5,2021-02-26,A102110,0.030000,K200,TIGER 200,EQ_M_KR_LARGE,41540.0,1256046.0,77215.429155,ETF


In [ ]:
def rebalance(assets, target_allocation, verbose=False):
    """
    Rebalances the portfolio using the specified target allocation, the portfolio's current allocation,
    and the available cash.
    Args:
        target_allocation (Dict[str, float]): Target asset allocation of the portfolio (in %). The keys of the dictionary are the tickers of the assets.
        verbose (bool, optional): Verbosity flag. Default is False. 
    Returns:
        (tuple): tuple containing:
            * new_units (Dict[str, int]): Units of each asset to buy. The keys of the dictionary are the tickers of the assets.
            * prices (Dict[str, [float, str]]): The keys of the dictionary are the tickers of the assets. Each value of the dictionary is a 2-entry list. The first entry is the price of the asset during the rebalancing computation. The second entry is the currency of the asset.
            * exchange_rates (Dict[str, float]): The keys of the dictionary are currencies. Each value is the exchange rate to CAD during the rebalancing computation.
            * max_diff (float): Largest difference between target allocation and optimized asset allocation.
    """

    # order target_allocation dict in the same order as assets dict and upper key
    target_allocation_reordered = {}
    try:
        for key in assets:
            target_allocation_reordered[key] = target_allocation[key]
    except:
        raise Exception(
            "'target_allocation not compatible with the assets of the portfolio."
        )

    target_allocation_np = np.fromiter(
        target_allocation_reordered.values(), dtype=float)

    assert abs(np.sum(target_allocation_np) -
               100.) <= 1E-2, "target allocation must sum up to 100%."

    # offload heavy work
    (balanced_portfolio, new_units, prices, cost, exchange_history) = rebalancing_helper.rebalance(self, target_allocation_np)

    # compute old and new asset allocation
    # and largest diff between new and target asset allocation
    old_alloc = self.asset_allocation()
    new_alloc = balanced_portfolio.asset_allocation()
    max_diff = max(
        abs(target_allocation_np -
            np.fromiter(new_alloc.values(), dtype=float)))

    if verbose:
        print("")
        # Print shares to buy, cost, new allocation, old allocation target, and target allocation
        print(
            " Ticker      Ask     Quantity      Amount    Currency     Old allocation   New allocation     Target allocation"
        )
        print(
            "                      to buy         ($)                      (%)              (%)                 (%)"
        )
        print(
            "---------------------------------------------------------------------------------------------------------------"
        )
        for ticker in balanced_portfolio.assets:
            print("%8s  %7.2f   %6.d        %8.2f     %4s          %5.2f            %5.2f               %5.2f" % \
            (ticker, prices[ticker][0], new_units[ticker], cost[ticker], prices[ticker][1], \
             old_alloc[ticker], new_alloc[ticker], target_allocation[ticker]))

        print("")
        print(
            "Largest discrepancy between the new and the target asset allocation is %.2f %%."
            % (max_diff))

        # Print conversion exchange
        if len(exchange_history) > 0:
            print("")
            if len(exchange_history) > 1:
                print(
                    "Before making the above purchases, the following currency conversions are required:"
                )
            else:
                print(
                    "Before making the above purchases, the following currency conversion is required:"
                )

            for exchange in exchange_history:
                (from_amount, from_currency, to_amount, to_currency,
                 rate) = exchange
                print("    %.2f %s to %.2f %s at a rate of %.4f." %
                      (from_amount, from_currency, to_amount, to_currency,
                       rate))

        # Print remaining cash
        print("")
        print("Remaining cash:")
        for cash in balanced_portfolio.cash.values():
            print("    %.2f %s." % (cash.amount, cash.currency))

    # Now that we're done, we can replace old portfolio with the new one
    self.__dict__.update(balanced_portfolio.__dict__)

    return (new_units, prices, exchange_history, max_diff)

In [33]:
old_alloc = easy_add_assets(tickers=tickers, quantities=quantities)

TypeError: easy_add_assets() missing 1 required positional argument: 'prices'

In [22]:
new_port.loc[:, ['exposure', 'wt']].to_dict('list')

{'exposure': ['FI_D_SHORT', 'MM', 'MM', 'FI_D_SHORT', 'MM', 'EQ_M_KR_LARGE'],
 'wt': [0.2499999999983805,
  0.24999999995716302,
  0.21785129484296648,
  0.1521487051029433,
  0.10000000000386107,
  0.029999999987254864]}

In [6]:
rootpath = '../../'
rel_path = './data/processed/'
filename = 'pa_weights.pkl'

In [7]:
weights = pd.read_pickle(rootpath+rel_path+filename)

In [8]:
rebal_date = '2021-02-26'  # 종목 가격 입수시점(현재는 종가 기준)

In [10]:
filename = 'balance_s.pkl'
balance_s = pd.read_pickle(rootpath+rel_path+filename)

In [125]:
balance_s

,date,userid,name,asset_class,itemcode,itemname,quantity,cost_price,cost_value,price,value,wt,group_by,투자원본
0,1/10/2020 1:00:00 PM,A01,투자자1,현금성,C000001,현금,700000,1,700000,1.00,700000.00,1.000000,20200110 13:00현금성,Y
1,1/10/2020 4:00:00 PM,A01,투자자1,현금성,C000001,현금,11055,1,11055,1.00,11055.00,0.015747,20200110 16:00현금성,N
2,1/10/2020 4:00:00 PM,A01,투자자1,현금성,D000001,예금,350000,1,350000,1.01,353500.00,0.503518,20200110 16:00현금성,N
3,1/10/2020 4:00:00 PM,A01,투자자1,주식,A069500,KODEX 200,1,42095,42095,41253.10,41253.10,0.058760,20200110 16:00주식,N
4,1/10/2020 4:00:00 PM,A01,투자자1,주식,A360200,KINDEX 미국S&P500,4,10865,43460,11082.30,44329.20,0.063142,20200110 16:00주식,N
5,1/10/2020 4:00:00 PM,A01,투자자1,채권,A196230,KBSTAR 단기통안채,1,104325,104325,100152.00,100152.00,0.142654,20200110 16:00채권,N
6,1/10/2020 4:00:00 PM,A01,투자자1,채권,A302190,TIGER 중장기국채,1,52365,52365,52365.00,52365.00,0.074588,20200110 16:00채권,N
7,1/10/2020 4:00:00 PM,A01,투자자1,대체,A329750,TIGER 미국달러단기채권액티브,4,9645,38580,10030.80,40123.20,0.057151,20200110 16:00대체,N
8,1/10/2020 4:00:00 PM,A01,투자자1,채권,A114260,KODEX 국고채3년,1,58120,58120,59282.40,59282.40,0.084441,20200110 16:00채권,N
9,2/11/2020 9:00:00 AM,A01,투자자1,현금성,C000001,현금,19565,1,19565,1.00,19565.00,0.027745,20200211 09:00현금성,N


In [11]:
# balance는 사용자(userid)의 최근 잔고내역.
balance = balance_s.loc[(balance_s.userid==userid) & (balance_s.date==balance_s.groupby(by='userid')['date'].max()[userid])]

In [12]:
balance

,date,userid,name,asset_class,itemcode,itemname,quantity,cost_price,cost_value,price,value,wt,group_by,투자원본
9,2/11/2020 9:00:00 AM,A01,투자자1,현금성,C000001,현금,19565,1,19565,1.00,19565.00,0.027745,20200211 09:00현금성,N
10,2/11/2020 9:00:00 AM,A01,투자자1,현금성,D000001,예금,350000,1,350000,1.01,353500.00,0.501298,20200211 09:00현금성,N
11,2/11/2020 9:00:00 AM,A01,투자자1,주식,A069500,KODEX 200,1,42095,42095,39990.25,39990.25,0.056710,20200211 09:00주식,N
12,2/11/2020 9:00:00 AM,A01,투자자1,주식,A360200,KINDEX 미국S&P500,2,10865,21730,10973.65,21947.30,0.031123,20200211 09:00주식,N
13,2/11/2020 9:00:00 AM,A01,투자자1,채권,A196230,KBSTAR 단기통안채,1,104325,104325,109541.25,109541.25,0.155340,20200211 09:00채권,N
14,2/11/2020 9:00:00 AM,A01,투자자1,채권,A302190,TIGER 중장기국채,1,52365,52365,52365.00,52365.00,0.074259,20200211 09:00채권,N
15,2/11/2020 9:00:00 AM,A01,투자자1,대체,A329750,TIGER 미국달러단기채권액티브,8,9645,77160,9355.65,74845.20,0.106138,20200211 09:00대체,N
16,2/11/2020 9:00:00 AM,A01,투자자1,주식,A367380,KINDEX 미국나스닥100,3,10920,32760,11138.40,33415.20,0.047386,20200211 09:00주식,N


In [85]:
old_port = balance.loc[:, ['itemcode', 'itemname', 'wt', 'value', 'quantity', 'price']]

In [100]:
access_path='../../data/processed/'
filename='advised_portfolios.pkl'
advised_pf = pd.read_pickle(access_path+filename)

In [83]:
from datetime import timedelta, datetime

In [84]:
balance_s.loc[(balance_s.userid==userid) & (balance_s.date==balance_s.groupby(by='userid')['date'].max()[userid])]

,date,userid,name,asset_class,itemcode,itemname,quantity,cost_price,cost_value,price,value,wt,group_by,투자원본
9,2/11/2020 9:00:00 AM,A01,투자자1,현금성,C000001,현금,19565,1,19565,1.00,19565.00,0.027745,20200211 09:00현금성,N
10,2/11/2020 9:00:00 AM,A01,투자자1,현금성,D000001,예금,350000,1,350000,1.01,353500.00,0.501298,20200211 09:00현금성,N
11,2/11/2020 9:00:00 AM,A01,투자자1,주식,A069500,KODEX 200,1,42095,42095,39990.25,39990.25,0.056710,20200211 09:00주식,N
12,2/11/2020 9:00:00 AM,A01,투자자1,주식,A360200,KINDEX 미국S&P500,2,10865,21730,10973.65,21947.30,0.031123,20200211 09:00주식,N
13,2/11/2020 9:00:00 AM,A01,투자자1,채권,A196230,KBSTAR 단기통안채,1,104325,104325,109541.25,109541.25,0.155340,20200211 09:00채권,N
14,2/11/2020 9:00:00 AM,A01,투자자1,채권,A302190,TIGER 중장기국채,1,52365,52365,52365.00,52365.00,0.074259,20200211 09:00채권,N
15,2/11/2020 9:00:00 AM,A01,투자자1,대체,A329750,TIGER 미국달러단기채권액티브,8,9645,77160,9355.65,74845.20,0.106138,20200211 09:00대체,N
16,2/11/2020 9:00:00 AM,A01,투자자1,주식,A367380,KINDEX 미국나스닥100,3,10920,32760,11138.40,33415.20,0.047386,20200211 09:00주식,N


In [73]:
risk_profile=2

In [86]:
new_port = advised_pf.loc[(advised_pf.risk_profile==risk_profile) & (advised_pf.date==advised_pf.groupby(by='risk_profile')['date'].max()[risk_profile])]

In [96]:
new_port = new_port.loc[:, ['itemcode', 'itemname', 'weights']]

In [97]:
pd.merge(old_port, new_port, left_on=['itemcode', 'itemname'], right_on=['itemcode', 'itemname'], how='outer')

,itemcode,itemname,wt,value,quantity,price,weights
0,C000001,현금,0.027745,19565.00,19565.0,1.00,NaN
1,D000001,예금,0.501298,353500.00,350000.0,1.01,NaN
2,A069500,KODEX 200,0.056710,39990.25,1.0,39990.25,NaN
3,A360200,KINDEX 미국S&P500,0.031123,21947.30,2.0,10973.65,NaN
4,A196230,KBSTAR 단기통안채,0.155340,109541.25,1.0,109541.25,NaN
5,A302190,TIGER 중장기국채,0.074259,52365.00,1.0,52365.00,NaN
6,A329750,TIGER 미국달러단기채권액티브,0.106138,74845.20,8.0,9355.65,NaN
7,A367380,KINDEX 미국나스닥100,0.047386,33415.20,3.0,11138.40,NaN
8,A122260,KOSEF 통안채1년,NaN,NaN,NaN,NaN,0.250000
9,A130730,KOSEF 단기자금,NaN,NaN,NaN,NaN,0.250000


In [102]:
new_port = new_port.rename(columns={'weights':'wt'})

In [103]:
new_port

,itemcode,itemname,wt
16051,A122260,KOSEF 통안채1년,0.250000
16052,A130730,KOSEF 단기자금,0.250000
16053,A153130,KODEX 단기채권,0.217851
16054,A114260,KODEX 국고채3년,0.152149
16055,A214980,KODEX 단기채권PLUS,0.100000
16056,A102110,TIGER 200,0.030000


In [104]:
pd.merge(old_port, new_port, left_on=['itemcode', 'itemname'], right_on=['itemcode', 'itemname'], how='outer', suffixes=['_old', '_new'])

,itemcode,itemname,wt_old,value,quantity,price,wt_new
0,C000001,현금,0.027745,19565.00,19565.0,1.00,NaN
1,D000001,예금,0.501298,353500.00,350000.0,1.01,NaN
2,A069500,KODEX 200,0.056710,39990.25,1.0,39990.25,NaN
3,A360200,KINDEX 미국S&P500,0.031123,21947.30,2.0,10973.65,NaN
4,A196230,KBSTAR 단기통안채,0.155340,109541.25,1.0,109541.25,NaN
5,A302190,TIGER 중장기국채,0.074259,52365.00,1.0,52365.00,NaN
6,A329750,TIGER 미국달러단기채권액티브,0.106138,74845.20,8.0,9355.65,NaN
7,A367380,KINDEX 미국나스닥100,0.047386,33415.20,3.0,11138.40,NaN
8,A122260,KOSEF 통안채1년,NaN,NaN,NaN,NaN,0.250000
9,A130730,KOSEF 단기자금,NaN,NaN,NaN,NaN,0.250000


In [7]:
from client import Client


NameError: name 'Singleton' is not defined